### Analysis of profitable app profiles in Google Play and App Store

In this project we'll try to find app profiles that are profitable in Google Play and App Store markets. As if we were Data Analysts for a company devoted to smartphone apps development, we want to provide the development team with insights regarding our products.


#### Preliminary data exploration

First of all, we want to open the data and explore it, in order to ...

Thus, what we'll do is open the CSV files and store the headers and the dataset themselves. Then, we can print the headers to discard those that are not relevant for our task, as well as print part of the dataset to know it better.

In [1]:
import csv

In [4]:
opened_googleplay_file = open('googleplaystore.csv')
googleplay_reader = csv.reader(opened_googleplay_file)
googleplay_csv = list(googleplay_reader)

opened_appstore_file = open('AppleStore.csv')
appstore_reader = csv.reader(opened_appstore_file)
appstore_csv = list(appstore_reader)    

googleplay_header, googleplay_dataset = googleplay_csv[0], googleplay_csv[1:]
appstore_header, appstore_dataset = appstore_csv[0], appstore_csv[1:]

explore_dataset(googleplay_dataset, 0, 5)
explore_dataset(appstore_dataset, 0, 5)

print(googleplay_header)
print(appstore_header)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']
['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']
['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']
['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']
['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


Number of rows: 10841
Number of columns: 13


['284882215', 'Facebook', '389879808', 'U

In [3]:
def explore_dataset(dataset, start, end):
    dataset_slice = dataset[start:end]    
    
    for row in dataset_slice:
        print(row)
    
    print('\n')
    print('Number of rows:', len(dataset))
    print('Number of columns:', len(dataset[0]))
    print('\n')

#### Data Cleaning

Before actually analysing data, we need to get rid of wrong data and rows that are not useful for us.

As stated [here](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015), the element in row 10472 is not correct. Therefore we'll remove it.

Moreover, there is duplicate data in our dataset (some apps appear more than once). In order to select the most recent data, we will take the "reviews" column into account (i.e.: we'll keep the row with the largest value for "reviews")

In [5]:
del googleplay_dataset[10472]

In [9]:
print(len(googleplay_dataset))

unique_apps_dict = dict()

for row in googleplay_dataset:
    if row[0] in unique_apps_dict:
        current_reviews = unique_apps_dict[row[0]][3]
        new_version_reviews = row[3]
        if new_version_reviews < current_reviews:
            continue
    unique_apps_dict[row[0]] = row
    
googleplay_dataset = list(unique_apps_dict.values())
        
print(len(googleplay_dataset))

9659
9659


In [12]:
from langdetect import detect

for row in googleplay_dataset:
    print('{}->{}'.format(row[0], detect(row[0])))

Photo Editor & Candy Camera & Grid & ScrapBook->en
Coloring book moana->tl
U Launcher Lite – FREE Live Cool Themes, Hide Apps->en
Sketch - Draw & Paint->en
Pixel Draw - Number Art Coloring Book->en
Paper flowers instructions->en
Smoke Effect Photo Maker - Smoke Editor->en
Infinite Painter->ro
Garden Coloring Book->nl
Kids Paint Free - Drawing Fun->en
Text on Photo - Fonteee->en
Name Art Photo Editor - Focus n Filters->en
Tattoo Name On My Photo Editor->en
Mandala Coloring Book->tl
3D Color Pixel by Number - Sandbox Art Coloring->en
Learn To Draw Kawaii Characters->en
Photo Designer - Write your name with shapes->en
350 Diy Room Decor Ideas->en
FlipaClip - Cartoon animation->tl
ibis Paint X->tl
Logo Maker - Small Business->no
Boys Photo Editor - Six Pack & Men's Suit->en
Superheroes Wallpapers | 4K Backgrounds->en
Mcqueen Coloring pages->fr
HD Mickey Minnie Wallpapers->en
Harley Quinn wallpapers HD->en
Colorfit - Drawing & Coloring->en
Animated Photo Editor->en
Pencil Sketch Drawing->en

join.me - Simple Meetings->fi
Invoice 2go — Professional Invoices and Estimates->en
Cisco Webex Meetings->en
ScreenMeet. Easy Phone Meeting->en
Cisco Webex Teams->en
Microsoft Remote Desktop->en
Start Meeting->en
ClickMeeting Webinars->de
BlueJeans for Android->da
Skype for Business for Android->no
Verify - Receipts & Expenses->de
LINE WEBTOON - Free Comics->en
Manga Master - Best manga & comic reader->en
GANMA! - All original stories free of charge for all original comics->en
Röhrich Werner Soundboard->de
Unicorn Pokez - Color By Number->en
MangaToon - Comics updated Daily->en
Manga Net – Best Online Manga Reader->de
Manga Rock - Best Manga Reader->id
Manga - read Thai translation->en
The Vietnam Story - Fun Stories->en
Dragon Ball Wallpaper - Ringtones->en
Funny Jokes Photos->fi
Truyện Vui Tý Quậy->vi
Comic Es - Shojo manga / love comics free of charge ♪ ♪->en
comico Popular Original Cartoon Updated Everyday Comico->en
Daily Manga - Comic & Webtoon->en
漫咖 Comics - Manga,Novel and Sto

FREE VIDEO CHAT - LIVE VIDEO AND TEXT CHAT->pt
Awake Dating->sw
Live Chat - Free Video Chat Rooms->en
Meet With Strangers: Video Chat & Dating->en
FREE LIVE TALK->vi
Random Chat App with Strangers->en
Ost. Zombies Cast - New Music and Lyrics->en
Dating White Girls->en
Live Chat - Free Video Talk->en
Geeks Dating->af
Live chat - free video chat->en
House party - live chat->en
Fishing Brain & Boating Maps Marine->en
CAM5678 Video Chat->en
Video chat live advices->en
chat live chat->en
Pet Lovers Dating->nl
Friend Find: free chat + flirt dating app->en
Latin Dating->tl
Spine- The dating app->en
Online Chat Girls Meet->en
Wifi Mingle->en
Girls Live Talk - Free Text and Video Chat->en
Soy Luna - Top Music And Lyrics->en
When Will You Get Married->en
HOW TO ASK A GIRL OUT->en
Online Girls Chat Group->en
The DJ - Match with People->en
Toronto Dating->en
i miss you quotes and photos->fr
JoJo Siwa Top Hits Music->en
UK Girls Mobile Numbers->de
chat saudi arabia->id
Chat Click - Dating Search->e

PTI Flex Maker, Photo Frame Editor & Songs 2018->en
Wipe out->fr
Birthdays & Other Events->en
Sarajevo Film Festival - Official->it
Summer Madness->da
Events High - Meet Your City!->en
vide-greniers.org->af
SUMMER SONIC app->en
Quake & Volcanoes: 3D Globe of Volcanic Eruptions->en
Prosperity->en
Picktrainer: India's largest photo contest app->en
AAS-IN-ASIA 2018->tl
Moot Peru 2018->id
FM News->en
Goldstar: Live Event Tickets->sv
Emmabodafestivalen->et
KudaGo - things to do in NY->en
Mindvalley U Tallinn 2018->et
Rockmaraton Info->en
Freitas Auctioneer Official->en
mobLee Events->nl
Xceed - Clubs, DJs, Festivals & Tickets->en
AMM Events & CPD->ca
Arab Halls - For Wedding & Events->de
MeAuDote - An act of love that saves a life->en
TCF National Conference->en
Endurance Lifestyle->fr
K PLUS->id
ING Banking->id
Citibanamex Movil->en
The postal bank->en
KTB Netbank->hu
Mobile Bancomer->it
Nedbank Money->nl
SCB EASY->de
CASHIER->en
Rabo Banking->tl
Capitec Remote Banking->en
Itau bank->id
Nu

Map My Ride GPS Cycling Riding->en
Daily Workouts - Exercise Fitness Routine Trainer->en
Google Fit - Fitness Tracking->en
Sworkit: Workouts & Fitness Plans->en
Map My Fitness Workout Trainer->en
Sports Tracker Running Cycling->en
Seven - 7 Minute Workout Training Challenge->en
Daily Yoga - Yoga Fitness Plans->en
Pocket Yoga->sv
Relax Meditation: Sleep with Sleep Sounds->en
Free Meditation - Take a Break->en
Meditate OM->ro
Yoga - Track Yoga->en
My Chakra Meditation->en
Relax with Andrew Johnson Lite->en
Meditation Music - Relax, Yoga->en
Meditation Studio->it
Down Dog: Great Yoga Anywhere->en
21-Day Meditation Experience->en
My Chakra Meditation 2->en
Simply Yoga - Fitness Trainer for Workouts & Poses->en
Calm - Meditate, Sleep, Relax->ro
Relax Melodies P: Sleep Sounds->af
Relax Melodies: Sleep Sounds->af
Simple Habit Meditation->it
Headspace: Meditation & Mindfulness->en
Yoga Studio: Mind & Body->en
Pregnancy & Baby Tracker->en
My Cycles Period and Ovulation->en
I’m Expecting - Pregn

...

Quran for Android->en
Al Quran Free - القرآن (Islam)->so
Taha Al Junaid Quran Mp3->id
Surah Al-Waqiah->so
Muslim Pro - Prayer Times, Azan, Quran & Qibla->id
Hisnul Al Muslim - Hisn Invocations & Adhkaar->et
I am rich->it
I AM RICH->en
I am Rich Plus->de
I am rich VIP->de
I Am Rich Premium->de
I am extremely Rich->en
I am Rich!->de
I am rich(premium)->sk
I am Rich Person->de
I am Rich Premium Plus->de
I Am Rich Pro->de
I Am Innocent->it
I am rich (Most expensive app)->en
i am rich->cy
I Am Rich->de
I am Bread->pt
Where Am I At?->en
I am Rich->de
Where Am I? GPS Loc->en
I am Millionaire - Richest guy in the town->en
Coast To Coast AM Insider->en
I AM RICH PRO PLUS->id
Where Am I?->en
I Am Wizard->pl
What Am I? - Brain Teasers->en
Who am I? (Biblical)->en
DWWW 774 Ultimate AM Radio->ro
Anno: Build an Empire->en
Vikings: an Archer's Journey->en
Trell: An app for Explorers | Best Wanderlust App->en
Virtual Families 2->es
My Emma :)->tl
League of Gamers - Be an E-Sports Legend!->en
Draw a St

Super Dancer->fr
AU Results - AU COE Corner->en
view.com.au - Buy, Rent or Research Real Estate->en
Au Weather Free->en
BringGo AU & NZ->en
AU Spark->id
Rugby AU->en
Learners Test Free: AU Driver Knowledge Test (DKT)->en
Au-allstar for KR->no
STUCOR - AU->de
loans.com.au Smart Money->fr
AU Students Zone (Result 2016)->de
Top #5 Bσσkie in AU->af
AU Results 2017 Auzone->ro
Au Pair->lt
AU Call Blocker - Block Unwanted Calls Texts 2018->en
realcommercial.com.au->it
Powershop AU->en
Australia Newspapers - AU News Apps->en
news.com.au->en
Media Sosial TNI AU->id
Dance On Mobile->fr
Super Dancer VN->da
Alerte au gogol->cy
Neuvaine au Saint-Esprit->fr
Neuvaines et prières au Coeur Sacré de Jésus->fr
SMS Au revoir->fr
allhomes.com.au->pt
Jobs in Canada - Emplois au Canada->pt
Predator Calls for Hunting AU->en
Romance et Mystères au Lycée->fr
Au Rubis les bijoutiers->fr
Love Dance->fr
mon guide au bank populaire -infos et instructions->fr
Avatar Musik->et
Job'Of - Emploi au Cameroun->pt
Mali 7 -

Railroad Radio Vancouver BC->en
AP Calculus BC: Practice Tests and Flashcards->en
Aura Hair Group BC->en
Bridge Constructor Playground FREE->en
BDCast - Bangla Live TV,Radio->en
bd's Mongolian Grill->en
BD Briight: Your Diabetes Personal Assistant->en
Jagobd - Bangla TV(Official)->en
Business Dictionary->en
BD Data Plan (3G & 4G)->id
বাংলা টিভি প্রো BD Bangla TV->bn
BD tools->et
BD Provider App->de
Izneo, Read Manga, Comics & BD->en
BD Radio TV Free List->en
BD Simplist Through Their Eyes->en
BD Online News->en
BD Bank Interest->de
BD Earn Pro->en
Comics Reader->en
Upohar BD->sv
Electricity Bill Calculator BD->en
BD Fishpedia->sq
BD Attendance->tr
BD All Sim Offer->de
BD TYCOON->tr
BD Sports Box & TV->en
Tour BD (Bahir Dar Map)->id
BD Field Force->en
BD CRICKET LIVE->en
BD All Results->ca
BD Online Passport Application->en
BD Internet Packages (Updated)->en
Live Radio BD->it
iCard BD Plus->en
BD Dialer->de
Youboox - Livres, BD et magazines->fr
BANGLA TV 3G/4G->de
Earn Money BD->en
BD H

BJJ Roadmap by Stephan Kesting->en
KISW 99.9 FM SEATTLE->de
BlackJack -21 Casino Card Game->en
Metal Detector Pro 2015->es
BlackJack 21 Pro->en
24 megapixel hd camera->ca
Ultimate BlackJack 3D FREE->en
BURGER KING® App->de
BURGER KING® MOBILE APP->de
BK Chat->en
BURGER KING® Puerto Rico->es
Free Coupons for Burger King->en
BK Manager->tl
Power Widget->pl
BK Murli Offline April 2017->ro
Read it easy for BK->en
BK Video Status->et
bk Group Mobile->en
SLANGY-Perfect BK World Theme->en
BK Formula Calculator 2->ro
Service Disabler->en
BK Photography->en
Keyboard Theme Dusk BK Purple->en
Coupons for Burger King->en
BK Formula Calculator->ro
VK->de
Korean English Translator->en
Dr Bk Sachin bhai->de
Audio Murli Glossary For BK->en
SKIN BK->de
BK Traffic Control cum Chart->en
Hungry Girl Diet Bk. Companion->en
Daily Murli Saar Widget->en
BK Suraj bhai->id
BK Shivani Videos->et
BK News Channel->en
BK Dinos->lt
Make a burger king->no
King Burger Dash->de
Official Madhuban Murli->en
Baba Yaad Hai

LangDetectException: No features in text.